# Introduction

Today, we have our first ever Kaggle Hack Session! We're going to be competing in the Titantic competition. The goal in this competition is to be able to predict who survived and who passed away during this tragedy, given information about the people involved.

We know that getting started for one of these competitions can be difficult, so we've provided this starter notebook to help you get up and running. Let's think about what we need to do when approaching any machine learning competition/problem. 

1) Determine your problem space. Do you have a classification problem, or a regression problem?

2) Determine what model you want to use (Always good to start off with simple models).

3) Load in and preprocess your dataset. Examine your database to see if there are any NULL or non-numeric values.

4) Split up your dataset into training and testing components. 

5) Create your model. This entails defining your function, your placeholders, the loss function, and the optimizer. 

6) Train, evaluate, and iterate on your model!

7) Once you have a model that you're satisfied with, load in test.csv (the test set for the Titanic competition), compute your predictions, save them to a CSV file, and submit to Kaggle. 

In [67]:
import pandas as pd
import tensorflow as tf

# Load in Data

You can download the data from the Kaggle website. The direct link is [here](https://www.kaggle.com/c/titanic/data), but we've already downloaded it for you. It's located in the Data subfolder. 

In [68]:
# Use the Pandas read_csv() function to load in the train.csv
titanicTrain = pd.read_csv('Data/train.csv')

# Examine Data

In [69]:
# Use the head function to see how the first couple rows of the dataframe looks like
titanicTrain.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [70]:
# Figure out what the different column names are
titanicTrain.columns.tolist()

['PassengerId',
 'Survived',
 'Pclass',
 'Name',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Ticket',
 'Fare',
 'Cabin',
 'Embarked']

Use other functions such as describe, max, mean, value_counts, etc to learn more about the dataset you're dealing with. 

# Clean Data

This is one of the most important parts of any machine learning pipeline. We want to make sure that the inputs we feed into any machine learning model are are valid, non-null, and are numerical values. To get you started with datapreprocessing, we'll show you one example of a column you may want to drop in this dataset 

In [71]:
# Visualize the data we're working with
titanicTrain.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB


So, as you can see above, some of the people don't have values for the age and cabin attributes. There are ways we can deal with this (for example, replace the null values with the median of the other values, replace them with 0, etc), but a simple method is to just drop the column.

In [72]:
# Drop the column
titanicTrain = titanicTrain.drop(['Cabin'], axis = 1) # For axis number (0 for rows and 1 for columns)

Another column that needs processing is the age.

In [73]:
# Do the preprocessing
medianAge = titanicTrain['Age'].median()
titanicTrain['Age'] = titanicTrain['Age'].fillna(medianAge)

Now, try it on your own! The functions you will probably be using are (although you're not limited to just these!):
- [drop()](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.drop.html)
- [fillna](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.fillna.html)
- [get_dummies()](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html)
- [dropna()](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.dropna.html)

In [74]:
# TODO Find the other attributes that may give us trouble later on! Once you find these
# columns, figure out if you just want to drop the attribute altogether or replace with 
# median, or something else!

# HINT: The name attribute is something you may want to look at. We don't want strings in our ML model!

Now that you know a couple ways of dealing with null values and string values, feel free to be creative! The best way to get a more accurate machine learning model is to understand the best ways to visualize and clean your data! This is one of the most important steps in any ML pipeline. 

# Create Training/Testing Matrices

So, now that we've made our final changes to our dataframe, we want to convert it into a matrix of numbers. We want our Y Matrix to be filled with binary labels indicating whether the person survived or not. Our X Matrix should contain all of the features that represent each individual.  

In [75]:
titanicTrain.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 11 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            891 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(4)
memory usage: 76.6+ KB


In [76]:
# Convert to matrices. 
# TODO Add/Remove columns as you see fit
X = titanicTrain[['Pclass', 'Age', 'SibSp', 'Parch', 'Fare']].as_matrix()
Y = titanicTrain['Survived'].as_matrix()
Y = Y.reshape([Y.shape[0], 1]) # Reshaping from (891,) to (891,1)
print (X.shape)
print (Y.shape)

(891, 5)
(891, 1)


(OPTIONAL) Remember that whenever we have a dataset, it's good practice to seperate the dataset into 2 parts, one that we will use to train the model, and one that we will use to check how our model is doing as a test/validation set. 

In [77]:
# Divide into xTrain, yTrain, xTest, and yTest. Take the last 100 examples as test
#numExamples = X.shape[0]
#numTestExamples = 100

#xTrain = X[:numExamples - numTestExamples] # xTrain contains the examples 0-791
#yTrain = Y[:numExamples - numTestExamples] # yTrain contains the labels for examples 0-791
#xTest = X[numExamples - numTestExamples:] # xTrain contains the examples 792-891
#yTest = Y[numExamples - numTestExamples:] # yTrain contains the labels for examples 792-891

# Create Model

Now that we have all of our data loaded in and preprocessed, we can start on creating our model. This component is pretty open ended. You have the freedom to choose whichever model you'd like to create. If you need inspiration, take a look at the code for linear regression and logistic regression in the week2 and week3 folders. A few other reminders:

- Think about what types of objects you'll need to create. Placeholders, variables, optimizers, etc
- Think back to how we created the linear regression and logistic regression models. 

In [79]:
# TODO Create your model here

# Train Model

Now that you've created your model by defining your computational graph, you're ready to start training the model. Remember that training model basically means that we want to run our optimizer object over different parts of our training dataset. A few other reminders:
- Remember to create a Tensorflow session and initialize all of your variables
- Run your optimizer object at every iteration
- Keep track of how your model is doing every now and again

In [82]:
numIterations = 1000 # Adjust this number as you see fit!
# TODO Create session and initialize variables
for i in range(numIterations):
    ...
    # TODO Run optimizer object over your data
    # TODO Check accuracy every once in a while

# Test Model

By now, you have a trained model and you're almost ready to submit! We want to now see how our model does on data that it has never seen before. We want to compute our predictions for the test set. We will then submit these predictions to Kaggle in order to see how accurate we are. A few reminders:
- Remember that preprocessing you did for the training dataset? You'll need to do that same preprocessing for this test set as well. 
- No need to initialize variables or anything. Everything is already trained! We just want to compute our predictions for this new set of data. 

In [83]:
titanicTest = pd.read_csv('Data/test.csv')

# TODO Do the same data preprocessing you did for the train set
# TODO Compute the predictions for the testing set by evaluating your logits/normalized logits variables
# TODO Check that the predictions are the correct dimensionality 

# Create Kaggle Submission

It's very important to be familiar with the exact Kaggle submission format. We basically want to create a CSV file where the first line of the CSV has the column names '' and '' (this will be different from competition to competition). The following lines will be contain the id number for the test as well as the prediction for that example.

In [66]:
import numpy as np
import csv

firstRow = [['id', 'pred']]
with open("predictions.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerows(firstRow)
    # TODO write the id number and the predictions you got from the last step!
    # HINT: Using a for loop might be helpful

Once you have the predictions.csv file, you can go ahead and submit to Kaggle! Great job!